In [ ]:
#setup

import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, ttest_ind, f_oneway
import numpy as np

sys.path.append(os.path.abspath(".."))
import Answercategories
print(dir(Answercategories))


sys.path.append(os.path.abspath(".."))
from Answercategories import (
    usefulness_order,
    deal_order,
    understanding_order,
    reliability_order,
    concerns_order,
    helpfulness_order
)



#load data
sys.path.append(os.path.abspath(".."))
from Analyse_umfrage import lade_excel

file_path = "c:/Users/tills/Bachelor-Code/Analyse_Files/Data/Fertige Tabelle.xlsx"
df = lade_excel(file_path)

# definition of questions to be analyzed

demographics = ["Gender", "Age", "Education Level", "Educational Level parent_1", "Educational Level parent_2", "CRT_points"]

survey_questions = [
    "Usefullness AI",
    "Deal with AI",
    "Understanding AI",
    "Frequency of use education",
    "Frequency of use everyday life",
    "Reliability AI",
    "Concerns AI",
    "Help of AI"
]


#helper function to test significance

def test_significance(df, question_col, demo_col):
    data = df[[question_col, demo_col]].dropna()

    if data[question_col].nunique() <= 5:
        contingency = pd.crosstab(data[demo_col], data[question_col])
        chi2, p, *_ = chi2_contingency(contingency)
        return p
    else:
        groups = [data[data[demo_col] == val][question_col] for val in data[demo_col].unique()]
        if len(groups) == 2:
            _, p = ttest_ind(*groups)
        else:
            _, p = f_oneway(*groups)
        return p

#analysis of each question

question_orders = {
    "Usefullness AI": usefulness_order,
    "Deal with AI": deal_order,
    "Understanding AI": understanding_order,
    "Reliability AI": reliability_order,
    "Concerns AI": concerns_order,
    "Help of AI": helpfulness_order
}



for question in survey_questions:
    print(f"\n Distribution of responses for: {question}")
    for demo in demographics:
        print(f"– Grouped by: {demo}")
        
        try:
            cross = pd.crosstab(df[demo], df[question], normalize='index') * 100
            #sort column based on defined order
            if question in question_orders:
                sort_order = question_orders[question]
                for col in sort_order:
                    if col not in cross.columns:
                        cross[col] = 0
                cross = cross[sort_order]

            #format output on one single line
            print(cross.round(1).to_string())

            #significance test
            p = test_significance(df, question, demo)
            significance = "✅ significant" if p < 0.05 else "❌ not significant"
            print(f"   → p-value: {p:.4f} → {significance}")
        
        except Exception as e:
            print(f" Test failed: {e}")



ModuleNotFoundError: No module named 'Answercategories'